## Try using aips and difmap coding to simulate Monte Carlo for antenna gain errors

### The operation here:
1. Generate gain facotors for each antenna at each time step(in python)
2. load and correct gain factors (aips, parseltongue)
3. export corrected data to difmap and run modelfit in difmap (python scripting)
4. Save results and analyze in python


In [1]:
import pexpect
import os
import sys
import re
import time
import numpy as np
import pandas as pd

In [2]:
def init_difmap(timeout=8000000):
    """初始化 difmap 会话并返回 difmap 对象和日志文件名"""
    difmap = pexpect.spawn('difmap')
    difmap.waitnoecho
    difmap.expect('0>')
    p = re.compile(r'(difmap.log_?\d*)')
    logfile = p.findall(difmap.before.decode())[0]
    difmap.timeout = timeout
    return difmap, logfile

def cleanup_difmap(difmap, logfile):
    """关闭 difmap 会话并删除日志文件"""
    difmap.sendline('quit')
    difmap.close()
    if os.path.isfile(logfile):
        os.remove(logfile)
        
def setup_mapsize(difmap, freq):
    """根据频率设置地图大小"""
    if freq <= 3:
        difmap.sendline('mapsize 2048,0.4')
    elif 3.1 <= freq <= 10:
        difmap.sendline('mapsize 2048,0.2')
    elif freq >= 10.1:
        difmap.sendline('mapsize 2048,0.1')
    difmap.expect('0>')

def prepare_observation(difmap, filename,file_exname, freq):
    """准备观测：加载文件，选择偏振，设置地图大小和 UV 权重"""
    uvf_file = filename + '.' + file_exname
    difmap.sendline('obs %s' % uvf_file)
    difmap.expect('0>')
    difmap.sendline('select i')
    difmap.expect('0>')
    setup_mapsize(difmap, freq)
    difmap.sendline('uvw 0,-2')
    difmap.expect('0>')

def getsnr_difmap(difmap):
    difmap.sendline('invert')
    difmap.expect('0>')
    difmap.sendline('print peak(flux,max)/imstat(rms)')
    difmap.expect('0>')
    # 修改正则表达式以支持科学计数法（如 7.333e-5）
    p = re.compile(r'([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?)')
    s = difmap.before.decode()
    print(p.findall(s))
    snr = float(p.findall(s)[0])
    difmap.sendline('print imstat(rms)')
    difmap.expect('0>')
    s2 = difmap.before.decode()
    rms = float(p.findall(s2)[0])
    difmap.sendline('print peak(x,max)')
    difmap.expect('0>')
    s3 = difmap.before.decode()
    peakx = float(p.findall(s3)[0])
    difmap.sendline('print peak(y,max)')
    difmap.expect('0>')
    s4 = difmap.before.decode()
    peaky = float(p.findall(s4)[0])
    return snr,rms,peakx,peaky

def iterative_modelfit(difmap, snr_threshold=5.5, max_iterations=12, model_type = 1):
    """迭代模型拟合：持续添加组件直到 SNR 低于阈值或达到最大迭代次数
    parm:
        difmap: the difmap progress from initi_difmap
        snr_threshold: the snr cut to decide how many iterations to go
        max_iteration: the maximum iterations. The progress will end if either of the above two parms reach the limit
        model_type: 0, delta; 1, gaussian; 2-4 not used, see difmap >help addcmp
    """
    snr, rms, pkx, pky = getsnr_difmap(difmap)
    print(snr, rms, pkx, pky)
    nm = 0
    while snr > snr_threshold:
        if nm >= max_iterations:
            print('limit reached, stop fitting')
            break
        difmap.sendline('addcmp 0.1,true,%f,%f,true,0,false,1,false,0,true,%i' % (pkx, pky, model_type))
        difmap.expect('0>')
        difmap.sendline('modelfit 80')
        difmap.expect('0>', timeout=500)
        snr, rms, pkx, pky = getsnr_difmap(difmap)
        print(snr, rms, pkx, pky)
        nm += 1
    return nm

def read_observation(difmap,filename):
    par_file = filename + '.par'
    difmap.sendline('@ %s' % par_file)
    difmap.expect('0>')
    

In [3]:
def parse_model_table(
        text: str, 
        default_freq=None
        ) -> pd.DataFrame:
    rows = []

    def to_float(tok: str) -> float:
        # 兼容偶尔出现的 0.123v 这种标记（这里一般不会出现在 East/North 表，但顺手处理）
        tok = tok.rstrip("v")
        try:
            return float(tok)
        except Exception:
            return np.nan

    for raw_line in text.splitlines():
        line = raw_line.strip()
        if not line:
            continue
        if line.startswith("#") or line.startswith("!") or line.startswith("-"):
            continue
        if "0>" in line:
            continue

        parts = line.split()

        # 有些 difmap 输出会带 component 编号（第一列是整数），这里做兼容
        comp_id = None
        if parts and parts[0].isdigit():
            comp_id = int(parts[0])
            parts = parts[1:]

        # East/North 那张表：最短到 theta_err 一共 15 列
        if len(parts) < 15:
            continue

        # 关键：只吃 East/North 那张表（第7列是 shape，且是纯字母，如 gauss/point）
        # 这能自动跳过你前面那行 "0.00346423v 0.141708v ... 1"
        if not (len(parts) > 6 and parts[6].isalpha()):
            continue

        row = {
            "component_id": comp_id,
            "flux_jy": to_float(parts[0]),
            "flux_err_jy": to_float(parts[1]),
            "x_arcsec": to_float(parts[2]),
            "x_err_arcsec": to_float(parts[3]),
            "y_arcsec": to_float(parts[4]),
            "y_err_arcsec": to_float(parts[5]),
            "type": parts[6],
            "ra_deg": to_float(parts[7]),
            "dec_deg": to_float(parts[8]),
            "major_fwhm_arcsec": to_float(parts[9]),
            "major_fwhm_err_arcsec": to_float(parts[10]),
            "minor_fwhm_arcsec": to_float(parts[11]),
            "minor_fwhm_err_arcsec": to_float(parts[12]),
            "theta_deg": to_float(parts[13]),
            "theta_err_deg": to_float(parts[14]),
            "freq_hz": np.nan,
            "spectral_index": np.nan,
            "spectral_index_err": np.nan,
        }

        # 如果后面真的有 freq/specIndex，就补上
        if len(parts) >= 16:
            row["freq_hz"] = to_float(parts[15])
        if len(parts) >= 17:
            row["spectral_index"] = to_float(parts[16])
        if len(parts) >= 18:
            row["spectral_index_err"] = to_float(parts[17])

        # 如果输出没带 freq，但你知道观测频率，就用 default_freq 补齐
        if (np.isnan(row["freq_hz"]) or row["freq_hz"] == 0.0) and default_freq is not None:
            row["freq_hz"] = float(default_freq)

        rows.append(row)

    df = pd.DataFrame(rows)

    # 如果 component_id 一列全是空，就删掉，表更干净
    if "component_id" in df.columns and df["component_id"].isna().all():
        df = df.drop(columns=["component_id"])

    return df

In [6]:

file_dir="/groups/public_cluster/home/ykzhang/VLBI/grb_data/bl307/calibrated_data_GRB221009a-v1/"
os.chdir(file_dir)
filename='161a_el20'

In [7]:
def get_model_parm(difmap, prompt=r"0>"):
    """
    在 difmap 里执行一条命令，并返回从命令回显到下一个提示符之间的全部文本输出。
    参数:
        difmap: difmap 进程对象
        prompt: difmap 提示符的正则表达式，默认为 "0>"
    返回:
        str: 命令输出的文本(text)
    """
    cmd = 'modelfit 0'
    difmap.sendline(cmd)
    difmap.expect(prompt)
    out = difmap.before or b""
    print(out)
    if isinstance(out, bytes):
        out = out.decode("utf-8", errors="replace")
    out = out.replace("\r", "")

    # difmap 通常会回显你输入的命令，把第一行是命令的情况去掉
    lines = out.splitlines()
    if lines and lines[0].strip() == cmd.strip():
        lines = lines[1:]
    return "\n".join(lines).strip()

In [8]:
freq = 2

difmap,difmap_log=init_difmap()
prepare_observation(difmap, filename, 'uvf', freq)
iterative_modelfit(difmap, snr_threshold=5.5, max_iterations=1, model_type = 1)
text = get_model_parm(difmap)
print(text)
df = parse_model_table(text)
cleanup_difmap(difmap, difmap_log)



['23.3747']
23.3747 0.000157677 0.0 0.0
['9.87249']
9.87249 7.08881e-05 -0.4 0.0
limit reached, stop fitting
b'modelfit 0\n\rPartitioning the model into established and variable parts.\r\nExtracting 1 model components from the UV plane model.\r\nThe fixed established model contains 0 components (0 Jy).\r\nThe variable part of the model contains 1 components (0.00346423 Jy).\r\nThere are 4 variables and 54186 usable visibilities.\r\nThis gives 2 x 54186 - 4 = 108368 degrees of freedom.\r\nReduced Chi-squared = Chi-squared / 108368.\r\n\r\nIteration 00: Reduced Chi-squared=0.99693117  Degrees of Freedom=108368\r\n! Flux (Jy) Radius (mas)  Theta (deg)  Major FWHM (mas)  Axial ratio   Phi (deg) T \\\r\n! Freq (Hz)     SpecIndex\r\n0.00346423v    0.141708v     37.9920v    0.220199      1.00000     0.00000v 1\r\n\r\n#    Flux (Jy)          East (arcsec)        North (arcsec)    Shape   R.A. (deg)       Dec (deg)  Major FWHM (arcsec) Minor FWHM (arcsec)  Theta (deg)     Freq (Hz)      Spectra

In [16]:
df

,flux_jy,flux_err_jy,x_arcsec,x_err_arcsec,y_arcsec,y_err_arcsec,type,ra_deg,dec_deg,major_fwhm_arcsec,major_fwhm_err_arcsec,minor_fwhm_arcsec,minor_fwhm_err_arcsec,theta_deg,theta_err_deg,freq_hz,spectral_index,spectral_index_err
0,0.003464,0.000068,0.000087,0.000005,0.000112,0.000014,gauss,288.264587,19.773397,0.00022,0.000019,0.00022,0.0,0.0,0.0,NaN,NaN,NaN


In [7]:
from pathlib import Path
file = Path("/Users/yingkangzhang/data/vlbi/vlbi_temp/161a_el20.fits")
dir = file.parent
os.chdir(dir)
os.makedirs("test_dir", exist_ok=True)

### 1. Generate gain factors for each antenna at each time step (in Python)

In [ ]:
def gen_antenna_gains(
    nants: int,
    gain_range: float = 0.1,
    dist: str = "uniform",
    seed: int | None = None,
) -> np.ndarray:
    """
    返回每根天线的幅度增益因子 g_i(real>0)。

    gain_range=0.1 -> uniform: [0.9, 1.1]
                      gaussian: mean=1, sigma=gain_range/2, 严格限定在 [0.9, 1.1]
    dist: "uniform" 或 "gaussian"   
    """
    rng = np.random.default_rng(seed)
    lo, hi = 1 - gain_range, 1 + gain_range

    if dist.lower() == "uniform":
        g = rng.uniform(lo, hi, size=nants)
    elif dist.lower() == "gaussian":
        sigma = gain_range / 2
        g = 1 + rng.normal(loc=0.0, scale=sigma, size=nants)
        g = np.clip(g, a_min=lo, a_max=hi)  # 严格限定在 [lo, hi] 范围内
    else:
        raise ValueError("dist must be 'uniform' or 'gaussian'")

    return g

In [ ]:
t = []
for i in range (10):
    g = gen_antenna_gains(8, gain_range=0.1, dist="gaussian", seed=None)
    t.append(g)
print(t)

[array([0.9763676 , 1.0164002 , 0.9883903 , 0.94875331, 1.01585807,
       1.0263361 , 1.02301885, 1.03927169]), array([0.97558872, 0.9512307 , 1.04055993, 0.9414121 , 1.04044525,
       0.97632577, 1.01796045, 1.00207396]), array([0.92068308, 0.92787154, 0.95531135, 0.9833696 , 0.95107106,
       1.01226005, 1.1       , 1.05468739]), array([0.92119516, 1.05095443, 1.04874986, 1.05093515, 0.99882777,
       0.97969162, 1.01892171, 0.95414144]), array([0.91318809, 1.00709351, 1.0052809 , 1.02656733, 0.96942443,
       1.04229469, 1.06214136, 1.01681327]), array([0.95475536, 0.96679453, 0.96954847, 1.01090309, 1.02313562,
       0.99665033, 1.09189165, 0.97684882]), array([0.97034512, 0.93203715, 1.1       , 0.98672983, 0.93517154,
       0.95824501, 0.95098042, 1.01499824]), array([0.97730366, 0.95187742, 1.00770207, 0.97729677, 0.96503017,
       0.98371663, 0.99481543, 1.05572956]), array([0.9       , 1.00218636, 1.01846571, 0.9546945 , 1.04148847,
       0.95918359, 1.03047067, 1.032

### 2. correct gain factors in aips

In [12]:
import sys
import os
# 获取notebook所在目录的父目录
sys.path.append('../vlbi-pipeline/')
from utils import gcal_apply

ModuleNotFoundError: No module named 'AIPS'

In [ ]:
tar_org_uvf = AIPSUVData(target_name,'uvf',1,1)
loadfr(filepath, filename, outname, outclass, outdisk, antname) # from run_tasks
tar_1_uvf = AIPSUVData(target_name+'_1','uvf',1,1)
copy_uvdata(indata, outname, outclass)
gcal_apply(indata,matx,cluse,pol)

run_split2(indata, source, gainuse, outclass, doband, bpver, flagver,av_chan, split_seq)